In [1]:
import yaml

%matplotlib inline
import sys, os
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")
import pcse
from pcse.models import Wofost72_PP
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse.fileinput import YAMLCropDataProvider
from pcse.util import WOFOST72SiteDataProvider, DummySoilDataProvider
from progressbar import printProgressBar

print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations

This notebook was built with:
python version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)] 
PCSE version: 5.5.5


In [2]:
# Define location, crop and season
latitude = 32.108136
longitude = -7.842794
crop_name = 'wheat'
variety_name = 'Winter_wheat_101'
campaign_start_date = '2022-01-01'
emergence_date = "2022-03-31"
harvest_date = "2022-10-20"
max_duration = 300

In [3]:
# Here we define the agromanagement for sugar beet
agro_yaml = """
- {start}:
    CropCalendar:
        crop_name: {cname}
        variety_name: {vname}
        crop_start_date: {startdate}
        crop_start_type: emergence
        crop_end_date: {enddate}
        crop_end_type: harvest
        max_duration: {maxdur}
    TimedEvents: null
    StateEvents: null
""".format(cname=crop_name, vname=variety_name, 
           start=campaign_start_date, startdate=emergence_date, 
           enddate=harvest_date, maxdur=max_duration)
agro = yaml.safe_load(agro_yaml)
print(agro_yaml)


- 2022-01-01:
    CropCalendar:
        crop_name: wheat
        variety_name: Winter_wheat_101
        crop_start_date: 2022-03-31
        crop_start_type: emergence
        crop_end_date: 2022-10-20
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null



In [4]:
# Weather data for Netherlands
wdp = NASAPowerWeatherDataProvider(latitude=latitude, longitude=longitude)

# Parameter sets for crop, soil and site
# Standard crop parameter library
cropd = YAMLCropDataProvider()

# We don't need soil for potential production, so we use dummy values
soild = DummySoilDataProvider()
# Some site parameters
sited = WOFOST72SiteDataProvider(WAV=50)

# Retrieve all parameters in the form of a single object. 
# In order to see all parameters for the selected crop already, we
# synchronise data provider cropd with the crop/variety: 
firstkey = list(agro[0])[0]
cropcalendar = agro[0][firstkey]['CropCalendar'] 
cropd.set_active_crop(cropcalendar['crop_name'], cropcalendar['variety_name'])
params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)

In [5]:
wofost = Wofost72_PP(params, wdp, agro)
wofost.run_till_terminate()

df_results = pd.DataFrame(wofost.get_summary_output())
#df_results = df_results.set_index("day")
df_results.tail()

,DVS,LAIMAX,TAGP,TWSO,TWLV,TWST,TWRT,CTRAT,RD,DOS,DOE,DOA,DOM,DOH,DOV,CEVST
0,0.0,5.089172,5097.160306,0.0,3313.154199,1784.006107,5097.160306,59.318003,120.0,None,2022-03-31,None,None,2022-10-20,None,10.291135


In [29]:
df_results_out = pd.DataFrame(wofost.get_output())
df_results_out = df_results_out.set_index("day")
df_results_out.tail()

,DVS,LAI,TAGP,TWSO,TWLV,TWST,TWRT,TRA,RD,SM,WWLOW
day,,,,,,,,,,,
2022-10-16,0.0,0.0,5097.160306,0.0,3313.154199,1784.006107,5097.160306,0.0,120.0,0.3,None
2022-10-17,0.0,0.0,5097.160306,0.0,3313.154199,1784.006107,5097.160306,0.0,120.0,0.3,None
2022-10-18,0.0,0.0,5097.160306,0.0,3313.154199,1784.006107,5097.160306,0.0,120.0,0.3,None
2022-10-19,0.0,0.0,5097.160306,0.0,3313.154199,1784.006107,5097.160306,0.0,120.0,0.3,None
2022-10-20,0.0,0.0,5097.160306,0.0,3313.154199,1784.006107,5097.160306,0.0,120.0,0.3,None


In [12]:
from pcse.fileinput import CABOFileReader

cropfile = os.path.join(data_dir, 'crop', 'SUG0601.crop')
cropd = CABOFileReader(cropfile)

In [4]:
soilfile = os.path.join(data_dir, 'soil', 'ec3.soil')
soild = CABOFileReader(soilfile)

In [5]:
from pcse.util import WOFOST72SiteDataProvider
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)

Unknown parameter values provided to WOFOSTSiteDataProvider: {'CO2': 360}
{'IFUNRN': 0, 'NOTINF': 0, 'SSI': 0.0, 'SSMAX': 0.0, 'WAV': 10.0, 'SMLIM': 0.4}


In [13]:
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

In [7]:
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

!!python/object/new:pcse.fileinput.yaml_agro_loader.YAMLAgroManagementReader
listitems:
- 2006-01-01:
    CropCalendar:
      crop_end_date: 2006-10-20
      crop_end_type: harvest
      crop_name: sugar-beet
      crop_start_date: 2006-04-05
      crop_start_type: emergence
      max_duration: 300
      variety_name: sugar-beet-601
    StateEvents: null
    TimedEvents: null



In [6]:
from pcse.fileinput import CSVWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'wdata.csv')
wdp = CSVWeatherDataProvider(weatherfile)
print(wdp)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
from pcse.fileinput import ExcelWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'wdata.xlsx')
wdp = ExcelWeatherDataProvider(weatherfile)
print(wdp)

In [32]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
wofsim = Wofost72_WLP_FD(parameters, data, agromanagement)

TraitError: The 'weatherdataprovider' trait of a Wofost72_WLP_FD instance expected a WeatherDataProvider or None, not the DataFrame             tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2020-01-01   9.4   6.8  12.0   0.0   0.0  245.0  19.1  68.0  1006.1   NaN
2020-01-02   5.0   3.1   6.8   0.6   0.0   85.0  18.5  61.0  1016.9   NaN
2020-01-03   7.6   4.4  10.8   4.9   0.0  123.0  28.5  55.0  1007.5   NaN
2020-01-04   7.3   5.6   8.9   2.5   0.0  185.0  21.8  54.0  1019.3   NaN
2020-01-05   5.2   3.3   7.0   1.4   0.0  137.0  23.5  72.0  1021.6   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2020-12-27   6.2   5.6   6.9   5.8   0.0   71.0  10.8   NaN  1019.1   NaN
2020-12-28   5.4   4.1   6.6   1.3   0.0  314.0  13.1  39.0  1020.9   NaN
2020-12-29   4.3   3.2   5.4   7.1   0.0   83.0  12.3  32.0  1023.9   NaN
2020-12-30   5.2   3.7   6.7  39.8   0.0   78.0  25.6 

In [15]:
wofsim.run_till_terminate()
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

,DVS,LAI,TAGP,TWSO,TWLV,TWST,TWRT,TRA,RD,SM,WWLOW
day,,,,,,,,,,,
2006-10-16,2.0,1.048787,21557.705173,14410.185356,3408.824045,3738.695772,2805.686837,0.052878,120.0,0.283340,34.000845
2006-10-17,2.0,1.055901,21628.846417,14474.212476,3412.381108,3742.252834,2807.138699,0.030516,120.0,0.282876,33.945161
2006-10-18,2.0,1.049414,21658.771700,14501.145230,3413.877372,3743.749098,2807.749419,0.030184,120.0,0.284605,34.152545
2006-10-19,2.0,1.052133,21685.964928,14525.619135,3415.237033,3745.108759,2808.304383,0.043053,120.0,0.284840,34.180768
2006-10-20,2.0,1.043584,21717.141971,14553.678474,3416.795885,3746.667611,2808.940650,0.039061,120.0,0.284271,34.112520


In [7]:
from get_market_value import *

agrom = {
    'latitude' : 37.5453,
    'longitude' : -7.6765,
    'crop_area' : 12,
    'crop_name' : "wheat",
    'variety_name' : "Winter_wheat_103",
    'campaign_start_date' : "2022-01-01",
    'emergence_date' : "2022-03-31",
    'harvest_date' : "2022-12-20",
    'max_duration' : 300}


print(get_market_value(agrom), "$")

17442.487659223007 $


In [13]:
import glob
import pandas as pd

data_path = 'data'

csv_files = sorted(glob.glob(data_path + '\\' + '*.csv'))

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate over each CSV file
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Append the DataFrame to the combined_df
    combined_df = combined_df.append(df, ignore_index=True)

# Print the combined DataFrame
print(combined_df)


Empty DataFrame
Columns: []
Index: []


In [1]:
import ee
import geemap

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

countyBoundary = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq('STATEFP', '55')) \
    .filter(ee.Filter.eq('NAME', 'Dane'))

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')
Map.centerObject(countyBoundary, 10)

Map.addLayer(countyBoundary, {'color': 'FF000000'}, 'County Boundary')

# Display the map
Map.addLayerControl()
Map

Map(center=[43.06727871132332, -89.41813275790595], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
from get_land_area import *

get_land_area(countyBoundary)

3199.4400446410014

In [4]:
import datetime as dt

start_date = dt.date(2022, 8, 20)

print(start_date)

2022-08-20


In [9]:
import pandas as pd
new_pd = pd.DataFrame(index = start_date.strftime('%m-%d'))

new_pd.loc[0] = 122

new_pd

TypeError: Index(...) must be called with a collection of some kind, '08-20' was passed